In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

#!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData 
ts = TimeSeries(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')
ti = TechIndicators(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')
fd = FundamentalData(key = 'HKDDDB1WO21X9QR6', output_format = 'pandas', indexing_type = 'date')

from functools import reduce

In [2]:
ticker = 'DOCU'

In [3]:
key = 'HKDDDB1WO21X9QR6'
base_url = "https://www.alphavantage.co/query/"

In [4]:
#build out a request
res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{ticker}',
    'apikey':key})

In [5]:
res.status_code

200

In [6]:
EPS = res.json()
# _, header = EPS.json()
#df = pd.DataFrame.from_dict(EPS, orient= 'index')_

In [7]:
len(EPS['quarterlyEarnings'])

11

In [8]:
def quarterly_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Quarterly
    for i in range(0,len(EPS['quarterlyEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True) 
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df['reportedDate'] = pd.to_datetime(df['reportedDate'])
    df[['reported_Quarterly_EPS', 'estimated_Quarterly_EPS', 'surprise', 'surprisePercentage']] = df[['reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = ['estimatedEPS', 'reportedEPS'])
    df = df.set_index('fiscalDateEnding')
    return df

In [9]:
df = quarterly_earnings_df(ticker)

In [10]:
df.dtypes

reportedDate               datetime64[ns]
surprise                          float64
surprisePercentage                float64
reported_Quarterly_EPS            float64
estimated_Quarterly_EPS           float64
dtype: object

In [11]:
reported_dict = pd.Series(df.reportedDate.values, index = df.index).to_dict()    

In [12]:
def annual_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Annual
    for i in range(0,len(EPS['annualEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True)
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df[['reported_Annual_EPS']] = df[['reportedEPS']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = 'reportedEPS')
    df = df.set_index('fiscalDateEnding')
    return df

In [13]:
df = annual_earnings_df(ticker)
#change index to the reported date for annual earnings 
#df.index = df.index.map(reported_dict)
df.head()

,reported_Annual_EPS
fiscalDateEnding,
2020-10-31,0.51
2020-01-31,0.31


In [14]:
def all_earnings_df(ticker):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    stock = ticker
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Quarterly
    for i in range(0,len(EPS['quarterlyEarnings'])-1):
        if i==0:
            df = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
        else:
            df_1 = pd.DataFrame.from_dict(EPS['quarterlyEarnings'][i], orient = 'index')
            df = pd.merge(df, df_1, left_index = True, right_index = True) 
    df = df.transpose()
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    df['reportedDate'] = pd.to_datetime(df['reportedDate'])
    df[['reported_Quarterly_EPS', 'estimated_Quarterly_EPS', 'surprise', 'surprisePercentage']] = df[['reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage']].apply(pd.to_numeric, errors = 'coerce')
    df = df.drop(columns = ['estimatedEPS', 'reportedEPS'])
    df = df.set_index('fiscalDateEnding')
    
    #create dictionary with fiscaldateending (key) and reporteddate (value) to map onto annual 
    reported_dict = pd.Series(df.reportedDate.values, index = df.index).to_dict()    
    
    #change index of quarterly to reported
    df = df.set_index('reportedDate')
    
    #Annual 
    res = requests.get(url=base_url, params={
    "function":"EARNINGS",
    "symbol":f'{stock}',
    'apikey':key})
    EPS = res.json()
    #Annual
    for i in range(0,len(EPS['annualEarnings'])-1):
        if i==0:
            dfa = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
        else:
            dfa_1 = pd.DataFrame.from_dict(EPS['annualEarnings'][i], orient = 'index')
            dfa = pd.merge(dfa, dfa_1, left_index = True, right_index = True)
    dfa = dfa.transpose()
    dfa['fiscalDateEnding'] = pd.to_datetime(dfa['fiscalDateEnding'])
    dfa[['reported_Annual_EPS']] = dfa[['reportedEPS']].apply(pd.to_numeric, errors = 'coerce')
    dfa = dfa.drop(columns = 'reportedEPS')
    dfa = dfa.set_index('fiscalDateEnding')
    
    #map new index values onto annual so that date is the reported date (more relevant to price action)
    dfa.index = dfa.index.map(reported_dict)

    #merge
    merge = pd.merge(df, dfa, how ='left', left_index = True, right_index = True)
    return merge

In [15]:
df = all_earnings_df(ticker)

In [16]:
df.head()

,surprise,surprisePercentage,reported_Quarterly_EPS,estimated_Quarterly_EPS,reported_Annual_EPS
reportedDate,,,,,
2020-12-03,0.0853,63.3259,0.22,0.1347,0.51
2020-09-03,0.0922,118.5090,0.17,0.0778,NaN
2020-06-04,0.0187,18.4600,0.12,0.1013,NaN
2020-03-12,0.0659,121.8115,0.12,0.0541,0.31
2019-12-05,0.0731,198.1030,0.11,0.0369,NaN


In [17]:
#Create funtion with other functions to get both annual and quarterly earnings into one DF

def get_all_earnings(ticker):
    dfq = quarterly_earnings_df(ticker)
    reported_dict = pd.Series(dfq.reportedDate.values, index = dfq.index).to_dict()    
    dfq = dfq.set_index('reportedDate')
    dfa = annual_earnings_df(ticker)
    dfa.index = dfa.index.map(reported_dict)
    merge = pd.merge(dfq, dfa, how ='left', left_index = True, right_index = True)
    return merge

In [18]:
earnings = get_all_earnings(ticker)

In [19]:
earnings.head(20)

,surprise,surprisePercentage,reported_Quarterly_EPS,estimated_Quarterly_EPS,reported_Annual_EPS
reportedDate,,,,,
2020-12-03,0.0853,63.3259,0.2200,0.1347,0.51
2020-09-03,0.0922,118.5090,0.1700,0.0778,NaN
2020-06-04,0.0187,18.4600,0.1200,0.1013,NaN
2020-03-12,0.0659,121.8115,0.1200,0.0541,0.31
2019-12-05,0.0731,198.1030,0.1100,0.0369,NaN
2019-09-05,-0.0276,-73.4043,0.0100,0.0376,NaN
2019-06-06,0.0249,55.2106,0.0700,0.0451,NaN
2019-03-14,0.0548,1053.8462,0.0600,0.0052,NaN
2018-12-06,-0.2994,-1931.6129,-0.3149,-0.0155,NaN


In [20]:
earnings = earnings.sort_index()

In [21]:
#Export earnings data to data folder
earnings.to_csv(f'data/{ticker}_earnings.csv')

#### Get Daily Stock Data for Ticker along with Technical Indicators and IWO Index Values 

For Ticker:
- Open 
- High
- Low
- Close
- Volume

Techincal Indicators:
- 8 Day Exponential Moving Average
- 21 Day Exponential Moving Average
- 50 Day Simple Moving Average
- 200 Day Simple Moving Average

IWO Index:
- 5 Day Exponential Moving Average
- 10 Day Exponential Moving Average
- 150 Day Exponential Moving Average

In [22]:
# Get daily stock data
daily, meta_data = ts.get_daily(symbol = ticker, outputsize = 'full')
#UNCOMMENT TO MERGE EARNINGS DATA WITH PRICE ACTION ----- merge = pd.merge(daily, df, how ='outer', left_index = True, right_index = True)

#add IWO stock data for 5day EMA, 10day EMA and 150day EMA
data_5, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 5, series_type = 'close')
data_10, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 10, series_type = 'close')
data_150, meta_data = ti.get_ema(symbol = 'IWO', interval='daily', time_period = 150, series_type = 'close')
dfs = [data_5, data_10, data_150]
trend = reduce(lambda left, right: pd.merge(left, right, on='date', how = 'inner'), dfs)
trend.columns = ['IWO_5d_EMA', 'IWO_10d_EMA', 'IWO_150d_EMA']

#Inner merge so that we only get IWO data for when the ticker has price data as well (in case ticker recently public)
merge = pd.merge(daily, trend, how ='inner',left_index = True, right_index = True)
merge.head()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA
date,,,,,,,,
2021-01-22,253.02,257.1250,250.8800,255.15,1468536.0,312.7773,308.7280,248.9662
2021-01-21,257.68,257.6800,251.6500,253.94,1495684.0,311.0610,307.0653,248.0636
2021-01-20,260.82,262.4699,253.6950,255.31,2788219.0,310.5865,305.9665,247.2052
2021-01-19,251.80,260.3800,251.6400,256.57,3117730.0,308.6048,304.0591,246.3013
2021-01-15,251.35,256.4400,246.8897,249.28,2512615.0,306.3321,302.0388,245.4040


In [23]:
merge.tail()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA
date,,,,,,,,
2018-05-03,39.6900,42.56,38.81,41.96,2783655.0,188.6693,189.0070,183.1276
2018-05-02,39.8500,40.50,38.75,39.32,1251440.0,188.8780,189.1748,183.0588
2018-05-01,38.7000,39.89,37.85,39.45,1285573.0,188.6219,189.1269,182.9738
2018-04-30,39.9047,40.19,37.53,38.63,3312463.0,188.6843,189.2669,182.8997
2018-04-27,38.0000,40.89,37.00,39.73,16392446.0,189.3126,189.6756,182.8389


In [24]:
# Create list of parameter dictionaries
def build_params(ticker):
    param_list = [{'EMA_8' : {
    "function":"EMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "8",
    "series_type": "high",
    'apikey':key}}, 
 
    {'EMA_21':{
    "function":"EMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "21",
    "series_type": "high",
    'apikey':key}}, 
                        
    {'SMA_50':{
    "function":"SMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "50",
    "series_type": "high",
    'apikey':key}},
              
    {'SMA_200':{
    "function":"SMA",
    "interval":"daily",
    "symbol":f'{ticker}',
    "time_period": "200",
    "series_type": "high",
    'apikey':key}}
             
    ]
    return param_list

In [25]:
def get_technicals(stock):
    key = 'HKDDDB1WO21X9QR6'
    base_url = "https://www.alphavantage.co/query/"
    ticker = stock
    dfs = []
    cols = []
    param_list = build_params(ticker)
    for i in param_list:    
        for k, v in i.items():
            res = requests.get(url=base_url, params = v)
            res_dict = res.json()
            _, header = res.json()
            df = pd.DataFrame.from_dict(res_dict[header], orient= 'index')
            dfs.append(df)
            cols.append(k)
    df = pd.concat(dfs, axis = 1)
    df.columns = cols
    df = df.apply(pd.to_numeric, errors = 'coerce')
    return df
            

In [26]:
technicals = get_technicals(ticker)
stock_data = pd.merge(merge, technicals, how ='inner',left_index = True, right_index = True)

In [27]:
stock_data.tail()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA,EMA_8,EMA_21,SMA_50,SMA_200
2018-05-14,44.99,45.7265,43.370,44.19,1032010.0,194.7091,193.2426,184.0986,44.2621,NaN,NaN,NaN
2018-05-11,44.56,45.8895,44.010,44.37,762193.0,194.3656,192.7640,183.9470,43.8436,NaN,NaN,NaN
2018-05-10,44.88,46.6990,43.660,44.01,1467879.0,193.4530,192.0025,183.7826,43.2591,NaN,NaN,NaN
2018-05-09,44.44,44.8700,43.245,44.10,1178122.0,192.3441,191.1873,183.6231,42.2763,NaN,NaN,NaN
2018-05-08,41.50,43.6799,40.500,43.22,821183.0,191.3039,190.4679,183.4781,41.5352,NaN,NaN,NaN


In [28]:
stock_data.head()

,1. open,2. high,3. low,4. close,5. volume,IWO_5d_EMA,IWO_10d_EMA,IWO_150d_EMA,EMA_8,EMA_21,SMA_50,SMA_200
2021-01-22,253.02,257.1250,250.8800,255.15,1468536.0,312.7773,308.7280,248.9662,256.3429,249.1721,235.9813,197.4756
2021-01-21,257.68,257.6800,251.6500,253.94,1495684.0,311.0610,307.0653,248.0636,256.1194,248.3768,235.1591,196.6452
2021-01-20,260.82,262.4699,253.6950,255.31,2788219.0,310.5865,305.9665,247.2052,255.6736,247.4465,234.7185,195.8092
2021-01-19,251.80,260.3800,251.6400,256.57,3117730.0,308.6048,304.0591,246.3013,253.7318,245.9442,234.3891,194.9265
2021-01-15,251.35,256.4400,246.8897,249.28,2512615.0,306.3321,302.0388,245.4040,251.8323,244.5006,233.8883,194.0943


In [29]:
#Sort index by datetime
stock_data = stock_data.sort_index()

In [30]:
#Export stock data to data folder
stock_data.to_csv(f'data/{ticker}_stock_data.csv')